# Tutorial: training FARE and E-FARE models on the Adult dataset

In this notebook, we will show how to use the FARE method to generate counterfactual interventions for the Adult dataset ([Dua and Graff (2019)](https://archive.ics.uci.edu/ml/citation_policy.html)). We will explain how to train and perform inference with FARE. For this notebook, we will use a support vector machine (SVM) with an RBF kernel as the black-box model. In the paper, we used instead a trained MLP, which offers a more challenging scenario. For more additional details, you can find the original paper below:

[De Toni, G., Lepri, B. & Passerini, A. Synthesizing explainable counterfactual policies for algorithmic recourse with program synthesis. Mach Learn (2023)](https://link.springer.com/article/10.1007/s10994-022-06293-7)

**If have any questions or if you spot any issue with the following notebook, you can reach me at [giovanni.detoni@unitn.it](giovanni.detoni@unitn.it)**

## How to install the library

You can easily install this library thorugh pip. We suggest using Python >= 3.7 and a virtualenv (e.g., conda). In this notebook we assume to have already a suitable conda environment with all the dependencies needed.

```bash
pip install git+https://github.com/unitn-sml/recourse-fare.git@v0.1.0
```

In [62]:
# We import the SVC class and some additional preprocessing methods
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_openml

# We just need to import the FARE model from rl_mcts
from recourse_fare.models.FARE import FARE
from recourse_fare.models.EFARE import EFARE

import pandas as pd
import numpy as np

import random
import torch

In [63]:
# Set some random seeds to ensure reproducibility
random.seed(2023)
np.random.seed(2023)
torch.manual_seed(2023)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Data preprocessing

We first retrieve the Adult dataset and complete some data cleaning activity. We first split the dataset into training and testing, and then we mainly perform two actions:
* We convert the `target` variable to either 0 (`>50K`) or 1 (`<=50K`);
* We replace unknown entries with `?` with the most frequent element for that given column. The most frequent element is taken by looking at the training set only and then by using those values to input the test set;

In [64]:
# We fetch the adult dataset from openml repository
data = fetch_openml(name='adult', version=2)
X = data.get("data").copy()
X["target"] = data.get("target").values

# Drop NaNs in the dataset
X.dropna(inplace=True)

# We drop some columns we do not consider actionable. It makes the problem less interesting, but it does
# show the point about how counterfactual interventions works. 
X.drop(columns=["fnlwgt", "age", "education-num", "race", "sex", "native-country", "relationship"], inplace=True)

y = X.target.apply(lambda x: 1 if x=="<=50K" else 0)
X.drop(columns=["target"], inplace=True)

In [65]:
# Split the dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

In [66]:
# Compute the most frequent attributes for the features with '?' values.
# We compute the most frequent attributes by looking at the training set only.
attrib, counts = np.unique(X_train['workclass'], return_counts = True)
most_freq_attrib_w = attrib[np.argmax(counts, axis = 0)]

attrib, counts = np.unique(X_train['occupation'], return_counts = True)
most_freq_attrib_o = attrib[np.argmax(counts, axis = 0)]

In [67]:
# We define a simple utility function which replaces the '?' with custom values
def clean(data, most_freq_attrib_o, most_freq_attrib_w, most_freq_attrib_c):
    data['occupation'] = data['occupation'].apply(lambda x: most_freq_attrib_o if x=='?' else x)
    data['workclass'] = data['workclass'].apply(lambda x: most_freq_attrib_w if x=='?' else x)
    return data

In [68]:
# We apply the clean() function both to the training and test set.
X_train = clean(X_train, most_freq_attrib_o, most_freq_attrib_w, None)
X_test = clean(X_test, most_freq_attrib_o, most_freq_attrib_w, None)

### Preprocessing Pipeline with scikit-learn

We then build a preprocessor to standardize/encode the various features. Namely, we use a `StandardScaler` and an `OneHotEncoder` to manage the real and categorical features, respectively. We exploit the `ColumnTransformer` class of scikit-learn to build a complete preprocessing pipeline.

In [69]:
# Build a preprocessing pipeline, which can be used to preprocess
# the elements of the dataset.
cat_selector = make_column_selector(dtype_include=[object, "category"])
num_selector = make_column_selector(dtype_include=np.number)
preprocessor = make_column_transformer(
    (StandardScaler(), num_selector), (OneHotEncoder(handle_unknown="ignore",sparse=False), cat_selector)
)

In [70]:
# Fit the preprocessor on the training data 
preprocessor.fit(X_train);

## Train and evaluating the black-box model

We train a simple `SVC` class in a "balanced" mode, where the misclassification errors are weighted by the relative numerosity of a class. Adult is an unbalanced dataset, thus we need to make sure that our model has a decent F1 score rather than just looking at the accuracy.


In [71]:
# Fit a simple SVC model over the data
blackbox_model = SVC(class_weight="balanced")
blackbox_model.fit(preprocessor.transform(X_train), y_train)

# Evaluate the model and print the classification report for the two classes
output = blackbox_model.predict(preprocessor.transform(X_test))
print(classification_report(output, y_test))

              precision    recall  f1-score   support

           0       0.87      0.55      0.67      3392
           1       0.78      0.95      0.86      5653

    accuracy                           0.80      9045
   macro avg       0.82      0.75      0.77      9045
weighted avg       0.81      0.80      0.79      9045



In [45]:
# Filter the training dataset by picking only the examples which are classified negatively by the model
output = blackbox_model.predict(preprocessor.transform(X_train))
X_train["predicted"] = output
X_train = X_train[X_train.predicted == 1]
X_train.drop(columns="predicted", inplace=True)

## FARE Model

Now comes the interesting part. We are going to show how to train the FARE model on the Adult dataset. The FARE model needs three different configurations parameters:
* **Policy Configuration**: it specifies how to build the internal agent. Please have a look at Figure 3 of the paper to understand the policy architecture.
* **Environment Configuration**: it specifies the environment where our agent will work on. Please have a look at the original implementation `recourse_fare/example/mock_adult_env.py` to understand its internal components.
* **MCTS Configuration**: it specifies some hyperparameters of the MCTS search component. 

In the cell below we show what we think are the most important configuration parameters for each component. 

In [46]:
policy_config= {
    "observation_dim": 47, # Size of the state's observation (after using the preprocessor defined above).
    "encoding_dim": 30, # Size of the output embedding of the state encoder. 
    "hidden_size": 30 # Size of the hiddel layers of the controller (LSTM).
}

environment_config = {
    "class_name": "recourse_fare.example.mock_adult_env.AdultEnvironment", # Class implementing the environment.
    "additional_parameters": {
        "preprocessor": preprocessor    # ColumnTransformer which is used to parse the environment. 
                                        # It is not a mandatory argument, but it is required by the AdultEnvironment class.
    }
}

mcts_config = {
    "number_of_simulations": 10, # How many simulations we want to perform at each MCTS node.
    "dir_epsilon": 0.3, # Parameter trading off exploration and exploitation (1.0 = only exploration).
    "dir_noise": 0.3 # Concentration parameter of the Dirichlet distribution used as "noise".
}

We build a FARE model by giving the constructor the `blackbox_model` and the configurations defined above. The `batch_size` indicates how many successful samples we need at each training step.

In [47]:
# Train a FARE model given the previous configurations
model = FARE(blackbox_model, policy_config, environment_config, mcts_config, batch_size=50)

### Train the FARE model

If you have done all correctly, training the FARE model is as easy as calling the `fit()` method as shown below. However, under certain conditions, the training accuracy might struggle to increase to a satisfactory level. Here we give some suggestions we learned during our experiments:
1. **Increase the `max_intervention_depth` value**. Such value indicates how long an intervention can be (e.g., how many actions the user might want to perform). Therefore, if a user needs to modify $N$ of its features to obtain recourse, a `max_intervention_depth` lower than $N$ might make the model unable to learn succesful interventions. As a rule of thumb, we suggest to set the `max_intervention_depth` value to at least $\frac{N}{2}$, where $N$ is the number of features.
2. **Improve the actions**. It might happen that the actions defined in the environment class might not be enough to obtain recourse. This situation might arise in two cases: either we are supplying to few arguments to an action (it means we might miss some potential good changes which could lead to recourse), or either the black-box classifier is making decisions based on non-actionable features (e.g., age, sex, native country etc.). In the latter case, there is little we can do, and it could be a hint that we trained an "unfair" model.
3. **Increase the `dir_epsilon` value**. Sometimes, it might happen that we do not provide enough noise to explore a potentially large actions space, thus, limiting the overall exploration.

If we specify the `tensorboard` argument, the FARE model will save training statistics in a directory. We can read them by using tensorboard. This feature is useful to check if the training is progressing correctly or if we need to take additional correction steps based on the suggestions above. If you want to display the information through tensorboard, then you need to run:
```
tensorboard --logdir ../notebooks/runs/adult
```

#### Note on the `max_iter` parameter

The `max_iter` parameter had a double meaning. It corresponds to how many potentially **different** users we sample from the training dataset and how many **training steps** we are performing. So, if we set 5000 as `max_iter`, we will sample at least 5000 users. However, we might perform fewer training steps (e.g., gradient updates). This behaviour depends on the `batch_size` parameter that indicates the minimum number of examples (sampled from the training buffer) we can use to train the model. Finding good interventions via MCTS might be hard and the training buffer would get slowly populated over each iterations. Therefore, we will not start training the underlying agent until we get `len(training_buffer) >= batch_size`     

In [48]:
# We fit the FARE model with tensorboard enabled.
# We need to make sure that the directory `../notebooks/runs/adult` exists
# since it is not created automatically.
model.fit(X_train, max_iter=500, tensorboard="../notebooks/runs/adult")

Train FARE:   0%|          | 0/500 [00:00<?, ?it/s]

In [49]:
# We save the trained FARE model to disk
model.save("/tmp/fare.pth")

# Load a pre-trained FARE model for the Adult dataset

In [50]:
# We load a pretrained model from a previous checkpoint
pretrained_model = "../notebooks/models/fare_adult-17_03_2023.pth"
model.load(pretrained_model)

In [51]:
# We use only the data which are negatively classified for testing
output = blackbox_model.predict(preprocessor.transform(X_test))
X_test["predicted"] = output
X_test = X_test[X_test.predicted == 1]
X_test.drop(columns="predicted", inplace=True)

Once we load the pretrained FARE model, we can run inference over the test instances. By specifying `full_output=True`, we fetch additional information from the model. Namely, we obtain the **counterfactual instances**, the **recourse results** (1 if we got recourse, 0 otherwise), the **counterfactual interventions** and the **costs** of those interventions.

In [72]:
# We run inference using FARE over 10 examples taken from the test set
counterfactuals, has_reached_recourse, traces, costs, _ = model.predict(X_test[0:100], full_output=True)

Eval FARE:   0%|          | 0/100 [00:00<?, ?it/s]

We now compute the $validity$, fraction of successful interventions we could find.

In [53]:
print(f"Validity: {sum(has_reached_recourse)/(len(has_reached_recourse))}")

Validity: 0.84


We can also extract an example of a counterfactual intervention (with both actions and arguments)

In [54]:
import pprint
print(pprint.pformat(traces[1]))

[['CHANGE_WORKCLASS', 'Private'],
 ['CHANGE_EDUCATION', 'Prof-school'],
 ['STOP', 0]]


## Train an E-FARE deterministic model

We now show how to train the deterministic model E-FARE, that will allow us to extract an automaton from the successful interventions. E-FARE also produces boolean rules that explain why the model suggested each of the actions in the intervention. The interface of the `EFARE` class is similar to the `FARE` API. For more the details of the E-FARE procedure, we point the reader to Section 3.6 and Section 3.7 of the paper.

### Data preprocessing for E-FARE

E-FARE trains a series of decision trees, one for each action available. If we want to have interpretable rules, we need to make our observations interpretable. For the Adult dataset, we keep the numerical values as they are, but we one hot encode the categorical values.    

In [55]:
cat_selector = make_column_selector(dtype_include=[object, "category"])
preprocessor_efare = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore",sparse=False), cat_selector), remainder="passthrough"
)

# We fit the EFARE preprocessor
preprocessor_efare.fit(X_train);

The E-FARE model needs only the pretrained FARE model and a preprocessor (which could be the same used by the FARE model) as arguments. Then, training it and saving it to disk are straightforward steps.

In [56]:
# We instantiate the EFARE model, we train it over 100 examples from the training set and we save it to disk.
efare_model = EFARE(model, preprocessor_efare)
efare_model.fit(X_train[0:100], verbose=True)
efare_model.save("/tmp/efare.pth")

Eval FARE:   0%|          | 0/100 [00:00<?, ?it/s]

[*] Compute rules given graph
[*] Getting rules for node INTERVENE
[*] Getting rules for node CHANGE_WORKCLASS
[*] Getting rules for node CHANGE_OCCUPATION
[*] Getting rules for node CHANGE_EDUCATION


In [57]:
# As we did for the FARE model, we load a pre-trained E-FARE model which will give us a decent validity
efare_model.load("../notebooks/models/efare_adult-17_03_2023.pth")

The `predict` API is the same as the `FARE` model. The only difference is that it returns the **rules** from the decision trees for each action. 

In [58]:
# We run inference using E-FARE
counterfactuals, has_reached_recourse, traces, costs, rules = efare_model.predict(X_test[0:100], full_output=True)

Eval EFARE:   0%|          | 0/100 [00:00<?, ?it/s]

We define a small function to parse the rules in an intelligible shape. It might be a bit cumbersome and depends on the preprocessing step. However, it is just for markup purposes.

In [73]:
# Function to clean up the rules extracted from EFARE
def clean_rules(rules):
    new_rule = []
    for single_rule in rules:
        tmp_rule = []
        for clause in single_rule:
            if "onehotencoder__" in clause:
                # This branch checks if the rule comes from an one-hot-encoded variable
                clause = clause.replace("onehotencoder__", "")             
                if "<= 0.5" in clause:
                    negation = "not"
                else:
                    negation = ""            
                clause = clause.replace("<= 0.5", "")
                clause = clause.replace("> 0.5", "")            
                feature, value = clause.rsplit('_', 1)            
                final_clause = negation+" "+feature+" = "+value
                tmp_rule.append(final_clause.strip())             
            elif "remainder__" in clause:
                clause = clause.replace("remainder__", "")    
                tmp_rule.append(clause)     
        new_rule.append(" and ".join(tmp_rule))
    return new_rule

Now that we have this function, we can easily use it to compute the results.

In [75]:
# We can print the rules for a given user
import pprint

print("Example of rules extracted by E-FARE:")
for action, rule in zip(traces[0], clean_rules(rules[0])):
    print(action, "\t", rule)

Example of rules extracted by E-FARE:
['CHANGE_EDUCATION', 'Prof-school'] 	 not workclass = State-gov and capital-loss <= 795.0 and not workclass = Federal-gov and education = Assoc-acdm and not marital-status = Divorced
['CHANGE_OCCUPATION', 'Tech-support'] 	 hours-per-week <= 42.5 and not occupation = Farming-fishing and not occupation = Sales and not occupation = Tech-support and hours-per-week <= 36.5
['STOP', 0] 	 education = Prof-school


At last, we compute the validity of EFARE. Clearly, we will get similar results to the FARE model. For the advantages of EFARE with respect to FARE, please have a look at the relevant section of the paper.

In [76]:
print(f"Validity: {sum(has_reached_recourse)/(len(has_reached_recourse))}")

Validity: 0.89
